In [286]:
import pandas as pd
import numpy as np
import re
import datetime as dt

PATH = '/Users/danielsagher/Dropbox/Documents/projects/bls_api_project/'
NOW = dt.datetime.now().strftime('%d-%b-%Y_%H:%M:%S')

nr = pd.read_csv(f'{PATH}outputs/main_op/national_results.csv')
sr = pd.read_csv(f'{PATH}outputs/main_op/state_results.csv')
ssd = pd.read_csv(f'{PATH}outputs/state_scrape_op/state_series_dimension.csv')
st = pd.read_csv(f'{PATH}outputs/excel_op/survey_table.csv')
nsd= pd.read_csv(f'{PATH}outputs/excel_op/national_series_dimension_og.csv')



In [287]:
# Checking for national_series_dimension duplicates

nsd['seriesID'].duplicated().sum()

6

In [288]:
# Checking which seriesID's are duplicated. Must be a manual entry error.

nsd[(nsd.seriesID.duplicated() == True)]

,series,seriesID,survey
199,Civilian Labor Force Level,LNS11000000,CPS
200,Civilian Labor Force Participation Rate,LNS11300000,CPS
201,Employment Level,LNS12000000,CPS
202,Employment - Population Ratio,LNS12300000,CPS
203,"Employed, Usually Work Full Time",LNS12500000,CPS
204,"Employed, Usually Work Part Time",LNS12600000,CPS


In [289]:
# Drop duplicates for national_series_dimension 

nsd = nsd.drop_duplicates()

In [290]:
# Re-checking duplicates

nsd.seriesID.duplicated().sum()

1

In [291]:
# Still a duplicate for just seriesID, which is pkey

nsd[(nsd.seriesID.duplicated() == True)]

,series,seriesID,survey
202,Employment - Population Ratio,LNS12300000,CPS


In [292]:
# Drop that duplicate

nsd.seriesID = nsd.seriesID.drop_duplicates(keep='first')

In [293]:
# Check SeriesID 

nsd.seriesID.duplicated().sum()

0

In [294]:
# Check for nulls

nsd[nsd.seriesID.isnull()]

,series,seriesID,survey
202,Employment - Population Ratio,NaN,CPS


In [295]:
# Drop Nulls

nsd.dropna(axis=0, inplace=True)

In [296]:
# Re-check

nsd[nsd.seriesID.isnull()]

,series,seriesID,survey


In [297]:
# Check datatypes

nr.dtypes

seriesID       object
year            int64
period         object
period_name    object
value          object
footnotes      object
dtype: object

In [298]:
# Replacing '-' with None and converting to float

nr['value'] = nr['value'].replace('-', None)
nr['value'] = nr['value'].astype('float')

nr.dtypes

seriesID        object
year             int64
period          object
period_name     object
value          float64
footnotes       object
dtype: object

In [299]:
# Replacing '-' with None and converting to float

sr['value'] = sr['value'].replace('-', None)
sr['value'] = sr['value'].astype('float')

sr.dtypes

seriesID        object
year             int64
period          object
period_name     object
value          float64
footnotes       object
dtype: object

In [300]:
# Check for spaces at beginning of strings

nsd[nsd.series == nsd.series.apply(lambda x: x.startswith(' '))],\
ssd[ssd.series == ssd.series.apply(lambda x: x.startswith(' '))]

(Empty DataFrame
 Columns: [series, seriesID, survey]
 Index: [],
 Empty DataFrame
 Columns: [survey, series, seriesID, state]
 Index: [])

In [301]:
# Remove spaces 

def space_remover(df):

        for col in df.columns:

                df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)
                
        return df


nr = space_remover(nr)
nsd = space_remover(nsd)
sr = space_remover(sr)
ssd = space_remover(ssd)
st = space_remover(st)
                

In [302]:
# Input new column is_adjusted (Boolean)

pattern1 = re.compile(', seasonally adjusted')
pattern2 = re.compile(', not seasonally adjusted')

nsd['is_adjusted'] = \
    nsd['series'].str.lower().apply(lambda x: True if re.search(pattern1, x) and not re.search(pattern2, x) else False if re.search(pattern2, x) else None)

ssd['is_adjusted'] = \
    ssd['series'].str.lower().apply(lambda x: True if re.search(pattern1, x) and not re.search(pattern2, x) else False if re.search(pattern2, x) else None)


In [303]:
# Remove seasonally adjusted and not seasonally adjusted from series 

def adjusted_remover(series):
    term1 = ', seasonally adjusted'
    term2 = ', not seasonally adjusted'
    
    def remove_terms(text):
        text_lower = text.lower()
        index1 = text_lower.find(term1)
        index2 = text_lower.find(term2)
        if index1 != -1:
            return text[:index1]
        elif index2 != -1:
            return text[:index2]
        return text
    
    # Apply the removal function to the 'series' column
    series['series'] = series['series'].apply(remove_terms)
    return series

# adjusted_remover(national_series_dimension)
nsd = adjusted_remover(nsd)
ssd = adjusted_remover(ssd)



In [304]:
# Check for 'seasonally adjusted'

nsd[(nsd['series'].str.lower().str.contains('seasonally adjusted') == True)].sum(),\
ssd[(ssd['series'].str.lower().str.contains('seasonally adjusted') == True)].sum()

(series         0
 seriesID       0
 survey         0
 is_adjusted    0
 dtype: object,
 survey         0
 series         0
 seriesID       0
 state          0
 is_adjusted    0
 dtype: object)

In [305]:
# Check for Booleans in national_series_dimension

nsd.is_adjusted.value_counts()

is_adjusted
True     41
False    11
Name: count, dtype: int64

In [306]:
# Check for Booleans in state_series_dimension

ssd.is_adjusted.value_counts()

is_adjusted
False    683
True     666
Name: count, dtype: int64

In [307]:
# Reorder columns

ssd = ssd[['seriesID', 'series', 'state','survey','is_adjusted']]

In [308]:
# Reorder columns

nsd = nsd[['seriesID', 'series', 'survey', 'is_adjusted']]

In [309]:
# Save to CSV in cleaned folder

nsd.to_csv(f'{PATH}outputs/cleaning_op/national_series_dimension_cleaned_{NOW}.csv', index=False)
ssd.to_csv(f'{PATH}outputs/cleaning_op/state_series_dimension_cleaned_{NOW}.csv', index=False)
sr.to_csv(f'{PATH}outputs/cleaning_op/state_results_cleaned_{NOW}.csv', index=False)
nr.to_csv(f'{PATH}outputs/cleaning_op/national_results_cleaned_{NOW}.csv', index=False)